Import BeatutifulSoup, request and pandas

In [86]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd

Load the webpage on python and parse html

In [87]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find_all('table')[0]

for the first table we have, which is the postal code table, load all the entry to a dataframe

In [174]:
# let's parse and add to the dataframe row by row
# make a temp table which has one row and collect data on this row.
tabletemp=[]
for row in table.find_all('tr'):
    rowtemp=[]
    for td in row.find_all('td'):
        rowtemp = rowtemp + [td.get_text()]
    tabletemp.append(rowtemp)
    headtemp=[]
for th in table.find_all('th'):
    headtemp= headtemp + [th.get_text()]
tabletemp[0]= headtemp

Get rid of all "\n" in the table

In [175]:
for eachrow in tabletemp:
    for i, word in enumerate(eachrow):
        eachrow[i] = word.strip('\n')

In [176]:
new_table = pd.DataFrame(tabletemp[1:],columns=tabletemp[0])

In [177]:
delete_index = new_table[new_table['Borough']=='Not assigned'].index
new_table.drop(delete_index, axis=0, inplace=True)

In [178]:
new_table.reset_index(drop=True, inplace=True)

In [207]:
neigh_na_index = new_table[new_table['Neighbourhood']=='Not assigned'].index
new_table.iloc[neigh_na_index, 2] = new_table[new_table['Neighbourhood']=='Not assigned']['Borough']

In [208]:
new_table=new_table.groupby(['Postcode','Borough'], as_index=False)['Neighbourhood'].agg(lambda nbh: ', '.join(nbh))

In [210]:
new_table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [209]:
new_table.shape

(103, 3)